In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 KB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp38-cp38-linux_x86_64.whl size=44089 sha256=9b3aa0aa55af83ce3a66c3ac1f40833f8a4d10bb929fcaffa670f2eac4dd3db6
  Stored in directory: /root/.cache/pip/wheels/52/dd/2b/10ff8b0ac81b93946bb5fb9e6749bae2dac246506c8774e6cf
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.2.1 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
Cloning into 'detectron2'...
remote: Enumerating objects: 14778, done.
remote: Cou

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 13.5 MB/s eta 0:00:00
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61431 sha256=c208ab92d6d46578b44c8a40f1943da767c04ce8d62ab69e98a11a8fce11a1d5
  Stored in directory: /root/.cache/pip/wheels/b8/79/07/c0e9367f5b5ea325e246bd73651e8af175fabbef943043b1cc
  Created wheel for antlr4-python3-runtime: filename=antlr4_python

In [ ]:
import torch, detectron2

TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cpu: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

torch:  1.13 ; cpu:  cu116
detectron2: 0.6


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random


# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
#utworzenie listy plików



from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
files=[]
import glob
for g in glob.glob(r"/content/gdrive/MyDrive/obrazki/10photo/*")[0:1]:
  files.append(g)
print(files)
#files zawiera ścieżki do obrazków
#wczytanie plików jako obiektów cv2.imread
im=[]
for x in files:
  im.append(cv2.imread(x))
#im zawiera wczytane obrazki

#Konfiguracja modelu detekcji

cfg = get_cfg()
cfg.MODEL.DEVICE = "cpu" 
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs_list=[]

#Detekcja obiektów
for x in im:
  
  outputs=predictor(x)
  outputs_list.append(outputs)
#outputs zawiera outputy z detekcji


#Funkcja usuwająca wszystkie outputy nie dotyczące klasy 'cars'
def onlykeeppersonclass(outputs):
  cls = outputs['instances'].pred_classes
  scores = outputs["instances"].scores
  masks = outputs['instances'].pred_masks
  #boxes = outputs['instances'].pred_boxes
  
  # remove all other classes which are not person(index:0)
  indx_to_remove = (cls != 2).nonzero().flatten().tolist()

  # delete corresponding arrays
  cls = np.delete(cls.cpu().numpy(), indx_to_remove)
  scores = np.delete(scores.cpu().numpy(), indx_to_remove)
  masks = np.delete(masks.cpu().numpy(), indx_to_remove, axis=0)
  #boxes = np.delete(boxes.cpu().numpy(), indx_to_remove, axis=0)

  # convert back to tensor and move to cuda
  cls = torch.tensor(cls).to('cpu:0')
  scores = torch.tensor(scores).to('cpu:0')
  masks = torch.tensor(masks).to('cpu:0')
  #boxes = torch.tensor(boxes).to('cuda:0')

  # not interested in boxes
  #outputs['instances'].remove('pred_boxes')

  # create new instance obj and set its fields
  obj = detectron2.structures.Instances(image_size=(aim.shape[0], aim.shape[1]))
  obj.set('pred_classes', cls)
  obj.set('scores', scores)
  obj.set('pred_masks', masks)
  return obj


newoutputs=[]
i=0

#Pętla tworząca liste outputów jedynie dla wykrytych obiektów klasy 'cars'
for x, y in zip(outputs_list, im):
  aim=y
  newoutputs.append(onlykeeppersonclass(x))
  i=i+1
  
print(newoutputs)

import ntpath

filenames=[]
for x in files:
    filenames.append(ntpath.basename(x))
print(filenames)

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
from skimage import measure
def close_contour(contour):
    if not np.array_equal(contour[0], contour[-1]):
        contour = np.vstack((contour, contour[0]))
    return contour
def binary_mask_to_polygon(binary_mask, tolerance=0):
    """Converts a binary mask to COCO polygon representation
    Args:
    binary_mask: a 2D binary numpy array where '1's represent the object
    tolerance: Maximum distance from original points of polygon to approximated
    polygonal chain. If tolerance is 0, the original coordinate array is returned.
    """

    polygons = []
    # pad mask to close contours of shapes which start and end at an edge
    padded_binary_mask = np.pad(binary_mask, pad_width=1, mode='constant', constant_values=0)
    contours = measure.find_contours(padded_binary_mask, 0.5)
    contours = np.subtract(contours, 1)
    for contour in contours:
        contour = close_contour(contour)
        contour = measure.approximate_polygon(contour, tolerance)
        if len(contour) < 3:
            continue
        contour = np.flip(contour, axis=1)
        segmentation = contour.ravel().tolist()
        # after padding and subtracting 1 we may get -0.5 points in our segmentation
        segmentation = [0 if i < 0 else i for i in segmentation]
        polygons.append(segmentation)
     
    return polygons
#Funkcja zamieniająca standardowy, binarny format masek obiektów, na format poligonalny 
#potrzebny przy odpalaniu skryptu podanego na moodle

def binary_mask_to_polygon(binary_mask, tolerance=0):
    """Converts a binary mask to COCO polygon representation
    Args:
    binary_mask: a 2D binary numpy array where '1's represent the object
    tolerance: Maximum distance from original points of polygon to approximated
    polygonal chain. If tolerance is 0, the original coordinate array is returned.
    """

    polygons = []
    # pad mask to close contours of shapes which start and end at an edge
    padded_binary_mask = np.pad(binary_mask, pad_width=1, mode='constant', constant_values=0)
    contours = measure.find_contours(padded_binary_mask, 0.5)
    contours = np.subtract(contours, 1)
    for contour in contours:
        contour = close_contour(contour)
        contour = measure.approximate_polygon(contour, tolerance)
        if len(contour) < 3:
            continue
        contour = np.flip(contour, axis=1)
        segmentation = contour.ravel().tolist()
        # after padding and subtracting 1 we may get -0.5 points in our segmentation
        segmentation = [0 if i < 0 else i for i in segmentation]
        polygons.append(segmentation)
     
    return polygons
polygonoutputs=[]
file_polygon=[]
#lista=list()
###Do naprawy zapisywanie masek w tym loopie
for x  in newoutputs:
    for y in x.pred_masks:
        
        
        polygonoutputs.append(binary_mask_to_polygon(y))
    file_polygon.append(polygonoutputs)
    i=i+1


i=0
j=0
k=0

k=0
l=0
#file_polygon[nr_pliku][nr_maski][nr_polygon]
##zamienic x,y na najpierw  x potem y  

k=0
l=0
#file_polygon[nr_pliku][nr_maski][nr_polygon]



for x,b in zip(file_polygon,filenames):#pliki
    
    dict={
            "version":detectron2.__version__ ,
            "flags":{},
            "shapes":[{"label":"car","points":[]}]

            
             }
    for y in x:#maski
             
        if(l!=0):dict["shapes"].append({"label":"car","points":[]})
        for z in y:#poligony
            print(z)
            for k in range(0,len(z)-1,2):
                 # dict["shapes"][l]["points"].append(list({z[k],z[k+1]}))
                dict["shapes"][l]["points"].append([z[k],z[k+1]])
            json_object = json.dumps(dict,indent=2)
            l=l+1
                    
                
      
                  
            
          
          
        with open(b+".json", "w") as outfile:
          outfile.write(json_object)
      
                  

Mounted at /content/gdrive/
['/content/gdrive/MyDrive/obrazki/10photo/50748171_1009685609223698_500716414173184000_n.jpg']
[01/26 08:17:01 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


model_final_f10217.pkl: 178MB [00:03, 48.2MB/s]                           
/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[Instances(num_instances=1, image_height=511, image_width=526, fields=[pred_classes: tensor([2]), scores: tensor([0.9850]), pred_masks: tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]]])])]
['50748171_1009685609223698_500716414173184000_n.jpg']
[224.0, 339.5, 223.0, 339.5, 222.0, 339.5, 221.0, 339.5, 220.0, 339.5, 219.0, 339.5, 218.0, 339.5, 217.0, 339.5, 216.0, 339.5, 215.0, 339.5, 214.0, 339.5, 213.0, 339.5, 212.0, 339.5, 211.0, 339.5, 210.0, 339.5, 209.0, 339.5, 208.0, 339.5, 207.0, 339.5, 206.0, 339.5, 205.0, 339.5, 204.0, 339.5, 203.0, 339.5, 202.0, 339.5, 201.5, 339.0, 201.0, 338.5, 200.0, 338.5, 199.0, 338.5, 198.0, 338.5, 197.5, 338.0, 197.0, 337.5, 196.0, 337.5, 1

[01/24 15:44:42 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


[Instances(num_instances=9, image_height=1600, image_width=2848, fields=[pred_classes: tensor([2, 2, 2, 2, 2, 2, 2, 2, 2]), scores: tensor([0.9966, 0.9937, 0.9933, 0.9903, 0.9897, 0.9870, 0.9818, 0.9455, 0.6854]), pred_masks: tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..

[1235.0, 1260.5, 1234.0, 1260.5, 1233.0, 1260.5, 1232.0, 1260.5, 1231.0, 1260.5, 1230.0, 1260.5, 1229.5, 1260.0, 1229.0, 1259.5, 1228.0, 1259.5, 1227.0, 1259.5, 1226.0, 1259.5, 1225.5, 1259.0, 1225.0, 1258.5, 1224.0, 1258.5, 1223.0, 1258.5, 1222.5, 1258.0, 1222.0, 1257.5, 1221.0, 1257.5, 1220.5, 1257.0, 1220.0, 1256.5, 1219.5, 1256.0, 1219.0, 1255.5, 1218.0, 1255.5, 1217.5, 1255.0, 1217.0, 1254.5, 1216.0, 1254.5, 1215.5, 1254.0, 1215.0, 1253.5, 1214.5, 1253.0, 1214.0, 1252.5, 1213.5, 1252.0, 1213.0, 1251.5, 1212.5, 1251.0, 1212.0, 1250.5, 1211.5, 1250.0, 1211.0, 1249.5, 1210.0, 1249.5, 1209.5, 1249.0, 1209.0, 1248.5, 1208.5, 1248.0, 1208.0, 1247.5, 1207.0, 1247.5, 1206.5, 1247.0, 1206.0, 1246.5, 1205.0, 1246.5, 1204.5, 1246.0, 1204.0, 1245.5, 1203.0, 1245.5, 1202.0, 1245.5, 1201.0, 1245.5, 1200.5, 1245.0, 1200.0, 1244.5, 1199.0, 1244.5, 1198.0, 1244.5, 1197.5, 1244.0, 1197.0, 1243.5, 1196.0, 1243.5, 1195.0, 1243.5, 1194.5, 1243.0, 1194.0, 1242.5, 1193.0, 1242.5, 1192.0, 1242.5, 1191.0,

In [ ]:
from PIL import Image, ImageDraw
import json


def read_points():
    f = open('/content/50748171_1009685609223698_500716414173184000_n.jpg.json')
    data = json.load(f)
    f.close()
    zbior_punktow = []
    for i in data['shapes']:
        punkty = []
        #print(i["label"])
        for j in i["points"]:
            #print(j)
            punkty.append((j[0], j[1]))
        zbior_punktow.append(punkty)
    return zbior_punktow
  



def read_img():
    img = Image.open("/content/gdrive/MyDrive/obrazki/10photo/50748171_1009685609223698_500716414173184000_n.jpg")
    w,h = img.size
    print("Format: ", img.format, "Rozmiar: ", img.size, img.mode)
    img1 = ImageDraw.Draw(img)  
    xy = read_points()
    #img1.polygon(xy,  outline ="red", fill ="#eeeeff")
    for p in xy:
        img1.polygon(p,  outline ="red") 
    img.show()
    img.save('final_image3.png')

if __name__ == '__main__':
    read_img()
    k = read_points()
    print("Liczba pinktow",len(k[0]))
    print("Wspolrzedne polygonu",k[0])


Format:  JPEG Rozmiar:  (526, 511) RGB
Liczba pinktow 1497
Wspolrzedne polygonu [(224.0, 339.5), (223.0, 339.5), (222.0, 339.5), (221.0, 339.5), (220.0, 339.5), (219.0, 339.5), (218.0, 339.5), (217.0, 339.5), (216.0, 339.5), (215.0, 339.5), (214.0, 339.5), (213.0, 339.5), (212.0, 339.5), (211.0, 339.5), (210.0, 339.5), (209.0, 339.5), (208.0, 339.5), (207.0, 339.5), (206.0, 339.5), (205.0, 339.5), (204.0, 339.5), (203.0, 339.5), (202.0, 339.5), (201.5, 339.0), (201.0, 338.5), (200.0, 338.5), (199.0, 338.5), (198.0, 338.5), (197.5, 338.0), (197.0, 337.5), (196.0, 337.5), (195.0, 337.5), (194.0, 337.5), (193.0, 337.5), (192.5, 337.0), (192.0, 336.5), (191.0, 336.5), (190.0, 336.5), (189.0, 336.5), (188.0, 336.5), (187.5, 336.0), (187.0, 335.5), (186.0, 335.5), (185.0, 335.5), (184.5, 335.0), (184.0, 334.5), (183.0, 334.5), (182.5, 334.0), (182.0, 333.5), (181.0, 333.5), (180.5, 333.0), (180.0, 332.5), (179.5, 332.0), (179.0, 331.5), (178.5, 331.0), (178.0, 330.5), (177.5, 330.0), (177.0,